In [150]:
# basic
import numpy as np
import pandas as pd

# get data
# ! conda install pandas-datareader s
import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
import matplotlib.pyplot as plt
import mpl_finance as mpf
%matplotlib inline
import seaborn as sns

#time
import datetime as datetime

# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib 
import talib

In [151]:
start_date = datetime.datetime(2018,1,1)     #開始日期
end_date= datetime.datetime(2018,12,31)      #結束日期

In [152]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

In [153]:
df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三個合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018
df_TWSE.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016/12/30,29.09,29.64,28.93,29.64,"6,121","213,121","2,939","3,692,175",20.56,1.23
1,1102 亞泥,2016/12/30,24.91,24.91,24.67,24.81,"3,347","88,078","1,227","3,361,447",31.75,0.73


In [154]:
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,"6,121","213,121","2,939","3,692,175",20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,"3,347","88,078","1,227","3,361,447",31.75,0.73


In [155]:
df_2327 = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
df_2456 = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
df_2478 = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
df_6271 = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE

In [157]:
df_2327.head(10)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2327 國巨,2016-01-04,68.75,68.75,67.33,68.62,"1,128","59,378",697,"651,594",9.38,1.37
1,2327 國巨,2016-01-05,68.1,68.49,67.33,68.36,"1,067","55,785",703,"651,594",9.34,1.36
2,2327 國巨,2016-01-06,68.36,69.4,67.84,68.1,"1,751","92,198",692,"651,594",9.31,1.36
3,2327 國巨,2016-01-07,66.81,68.62,66.81,68.1,992,"52,047",657,"651,594",9.31,1.36
4,2327 國巨,2016-01-08,68.1,69.27,67.71,69.27,"1,156","61,362",598,"651,594",9.47,1.38
5,2327 國巨,2016-01-11,68.75,69.14,68.23,68.75,576,"30,551",379,"651,594",9.4,1.37
6,2327 國巨,2016-01-12,68.75,69.4,68.75,69.27,757,"40,398",451,"651,594",9.47,1.38
7,2327 國巨,2016-01-13,69.27,69.4,68.1,68.23,675,"35,740",466,"651,594",9.33,1.36
8,2327 國巨,2016-01-14,67.71,69.01,67.2,68.49,852,"44,823",543,"651,594",9.36,1.36
9,2327 國巨,2016-01-15,68.49,69.01,67.46,67.71,"1,219","63,795",697,"651,594",9.26,1.35


In [ ]:
df_2327['Date']=df_2327.index             # index 複製到另一欄
df_2327 = df_2327.reset_index(drop=True)  # index變成 0~N
df_2327.head()

In [ ]:
##增欄位:fluctuation幅度 tag漲跌平
df_2327['fluctuation']=0
df_2327['tag']='--'
df_2327.head()

In [ ]:
###計算漲跌
for index, row in df_2327.iterrows():
    try:
        margin =(float(df_2327.loc[index,'High']) - float(df_2327.loc[index-1,'Close']) )/ float(df_2327.loc[index-1,'Close'])
        df_2327.loc[index,'fluctuation']=margin
        if margin >0.03:
            df_2327.loc[index,'tag']='up'
        elif margin<-0.03:
            df_2327.loc[index,'tag']='down'
    except:
        continue

In [ ]:
up_fliter=(df_2327['tag']=='up')
up_2327=df_2327[up_fliter]
up_2327.head()

In [ ]:
down_fliter=(df_2327['tag']=='down')
down_2327=df_2327[down_fliter]
down_2327

In [ ]:
downDate=down_2327["Date"].astype(str)
downDate

In [ ]:
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")

In [ ]:
df_forum[['post_time','title','content']]

In [ ]:
###開始做兩種資料表中的時間比對
forumDate=df_forum['post_time'].astype(str)
forumDate.head()